In [1]:
import csv
import math
import string
import itertools
from io import open
from conllu import parse_incr
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import wordnet as wn

In [2]:
# tests for pytorch computation graph
from torch.nn import CosineEmbeddingLoss
l = CosineEmbeddingLoss()
import torch.optim as optim

sense_vec = torch.ones(3, 1)
sense_vec.requires_grad = True
def_vec = torch.randn(3, 1)
def_vec.requires_grad = True

matrix = torch.ones(3, 3)
matrix.requires_grad = True
vec = matrix[2, :].view(3, 1)
print(vec.grad_fn)
print(vec.requires_grad)
label = torch.randn(3, 1)

opt = optim.Adam([matrix])
test_vec = torch.ones(def_vec.size())

for i in range(1):
    
    opt.zero_grad()
    loss = l(vec, label, test_vec)
    # print(loss)
    loss.backward()
    print(matrix.grad)
    # print(def_vec.grad)
    
    opt.step()

print(matrix)

True
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.9802e-08, 0.0000e+00, 0.0000e+00]])
tensor([[1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [0.9999, 1.0000, 1.0000]], requires_grad=True)


In [3]:
# parse the WSD dataset first

'''
Copyright@
White, A. S., D. Reisinger, K. Sakaguchi, T. Vieira, S. Zhang, R. Rudinger, K. Rawlins, & B. Van Durme. 2016. 
[Universal decompositional semantics on universal dependencies]
(http://aswhite.net/media/papers/white_universal_2016.pdf). 
To appear in *Proceedings of the Conference on Empirical Methods in Natural Language Processing 2016*.
'''

def parse_wsd_data():

    # parse the WSD dataset
    wsd_data = []

    # read in tsv by White et. al., 2016
    with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:

        tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')      

        # store the data: ordered dict row
        for row in tsv_reader:                                

            # each data vector
            wsd_data.append(row)

        # make sure all data are parsed
        print('Parsed {} word sense data from White et. al., 2016.'.format(len(wsd_data)))

    return wsd_data

In [4]:
# get the raw wsd data
wsd_data = parse_wsd_data()

Parsed 439312 word sense data from White et. al., 2016.


In [5]:
'''
return: 
all senses for each word 
all definitions for each word
all supersenses
from the EUD for train, test, and dev dataset
index provided by WSD dataset by White et. al.
'''
# get all the senses and definitions for each word from WSD dataset
# order of senses and definitions are in order
def get_all_senses_and_definitions(wsd_data):

    # all senses for each 
    all_senses = {}
    all_definitions = {}
    all_supersenses = {}
    
    # for test purpose: only load specific amount of data
    for i in range(len(wsd_data)):

        # get the original sentence from EUD
        sentence_id = wsd_data[i].get('Sentence.ID')
        
        # get the definitions for the target word from EUD
        definition = wsd_data[i].get('Sense.Definition').split(' ')
        
        # the index in EUD is 1-based!!!
        sentence_number = int(sentence_id.split(' ')[-1]) - 1
        word_index = int(wsd_data[i].get('Arg.Token')) - 1
        word_lemma = wsd_data[i].get('Arg.Lemma')
        word_sense = wsd_data[i].get('Synset')
        response = wsd_data[i].get('Sense.Response')
        
        # supersense-> (word_lemma, word_sense) dictionary
        super_s = wn.synset(word_sense).lexname().replace('.', '_')
        if all_supersenses.get(super_s, 'not_exist') != 'not_exist':
            all_supersenses[super_s].add((word_lemma, word_sense))
        else:
            all_supersenses[super_s] = {(word_lemma, word_sense)}

        # if the word already exits: add the new sense to the list
        # else: creata a new list for the word
        if all_senses.get(word_lemma, 'not_exist') != 'not_exist':
            if word_sense not in all_senses[word_lemma]:
                all_senses[word_lemma].append(word_sense)
        else:
            all_senses[word_lemma] = []
            all_senses[word_lemma].append(word_sense)            
            
        if all_definitions.get(word_lemma,'not_exist') != 'not_exist':
            if definition not in all_definitions[word_lemma]: 
                all_definitions[word_lemma].append(definition)
        else:
            all_definitions[word_lemma] = []
            all_definitions[word_lemma].append(definition)
        
    return all_senses, all_definitions, all_supersenses

In [6]:
# get all the senses and definitions
all_senses, all_definitions, all_supersenses = get_all_senses_and_definitions(wsd_data)


In [7]:
# test for the WordNet NLTK API
'''
The specific Synset method is lexname, e.g. wn.synsets('spring')[0].lexname(). 
That should make it really easy to get the suspersenses.
And if you have the synset name–e.g. 'spring.n.01'
you can access the supersense directly: wn.synset('spring.n.01').lexname().
Which returns 'noun.time'.
And wn.synset('spring.n.02').lexname() returns 'noun.artifact'

for idx, d in enumerate(all_definitions['spring']):
    print(d)
    print(wn.synset(all_senses['spring'][idx]).lexname())

for _ in wn.synsets('spring'):
    print(_.lexname())
'''

"\nThe specific Synset method is lexname, e.g. wn.synsets('spring')[0].lexname(). \nThat should make it really easy to get the suspersenses.\nAnd if you have the synset name–e.g. 'spring.n.01'\nyou can access the supersense directly: wn.synset('spring.n.01').lexname().\nWhich returns 'noun.time'.\nAnd wn.synset('spring.n.02').lexname() returns 'noun.artifact'\n\nfor idx, d in enumerate(all_definitions['spring']):\n    print(d)\n    print(wn.synset(all_senses['spring'][idx]).lexname())\n\nfor _ in wn.synsets('spring'):\n    print(_.lexname())\n"

In [8]:
# read the train, dev, test datasets from processed files
# check the 'data_loader.ipynb' for details
def read_file():
    
    train_X = []
    train_X_num = 0
    train_Y = []
    train_Y_num = 0
    test_X = []
    test_X_num = 0
    test_Y = []
    test_Y_num = 0
    dev_X = []
    dev_X_num = 0
    dev_Y = []
    dev_Y_num = 0
    
    train_word_idx = []
    test_word_idx = []
    dev_word_idx = []
    
    # read in csv
    with open('data/train_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            train_X.append(row)
            train_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {train_X_num} data points for train_X.')

    with open('data/train_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            train_Y.append(row)
            train_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {train_Y_num} data points for train_Y.')
        
    with open('data/train_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            train_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(train_word_idx)} data points for train_word_idx.')

    with open('data/dev_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            dev_X.append(row)
            dev_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {dev_X_num} data points for dev_X.')

    with open('data/dev_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            dev_Y.append(row)
            dev_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {dev_Y_num} data points for dev_Y.')
        
    with open('data/dev_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            dev_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(dev_word_idx)} data points for dev_word_idx.')
        
    with open('data/test_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            test_X.append(row)
            test_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {test_X_num} data points for test_X.')

    with open('data/test_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            test_Y.append(row)
            test_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {test_Y_num} data points for test_Y.')
        
    with open('data/test_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            test_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(test_word_idx)} data points for test_word_idx.')    
        
    return train_X, train_Y, test_X, test_Y, dev_X, dev_Y, train_word_idx, test_word_idx, dev_word_idx

In [35]:
train_X, train_Y, test_X, test_Y, dev_X, dev_Y, train_word_idx, test_word_idx, dev_word_idx = read_file()

# test on one word
word_choice = 'level'

new_train_X = []
new_train_Y = []
new_train_idx = []
distri_train = [0 for _ in range(len(all_senses[word_choice]))]
for index, sen in enumerate(train_X):
    
    if sen[train_word_idx[index]] == word_choice:
        new_train_idx.append(train_word_idx[index])
        new_train_X.append(sen)
        new_train_Y.append(train_Y[index])
        for i, response in enumerate(train_Y[index]):
            if response:
                distri_train[i] += 1
print('distri of train: {}'.format(distri_train))
        
new_test_X = []
new_test_Y = []
new_test_idx = []
distri_test = [0 for _ in range(len(all_senses[word_choice]))]
for index, sen in enumerate(test_X):
    
    if sen[test_word_idx[index]] == word_choice:
        new_test_idx.append(test_word_idx[index])
        new_test_X.append(sen)
        new_test_Y.append(test_Y[index])
        for i, response in enumerate(test_Y[index]):
            if response:
                distri_test[i] += 1
print('distri of test: {}'.format(distri_test))

new_dev_X = []
new_dev_Y = []
new_dev_idx = []
distri_dev = [0 for _ in range(len(all_senses[word_choice]))]
for index, sen in enumerate(dev_X):
        
    if sen[dev_word_idx[index]] == word_choice:
        new_dev_idx.append(dev_word_idx[index])
        new_dev_X.append(sen)
        new_dev_Y.append(dev_Y[index])
        for i, response in enumerate(dev_Y[index]):
            if response:
                distri_dev[i] += 1
print('distri of dev: {}'.format(distri_dev))

target_senses = all_senses[word_choice]
new_all_senses = {word_choice : target_senses}
# print(new_all_senses)
target_def = all_definitions[word_choice]
new_all_def = {word_choice : target_def}

new_all_supersenses = {}
for supersense in all_supersenses.keys():
    for tuples in all_supersenses[supersense]:
        
        if tuples[0] == word_choice:
            if new_all_supersenses.get(supersense, 'e') != 'e':
                new_all_supersenses[supersense].add((word_choice, tuples[1]))
            else:
                new_all_supersenses[supersense] = {(word_choice, tuples[1])}


Parsed 67398 data points for train_X.
Parsed 67398 data points for train_Y.
Parsed 67398 data points for train_word_idx.
Parsed 7333 data points for dev_X.
Parsed 7333 data points for dev_Y.
Parsed 7333 data points for dev_word_idx.
Parsed 7123 data points for test_X.
Parsed 7123 data points for test_Y.
Parsed 7123 data points for test_word_idx.
distri of train: [35, 43, 27, 11, 6, 3, 7, 5]
distri of test: [5, 5, 4, 1, 1, 1, 2, 1]
distri of dev: [4, 3, 0, 2, 0, 0, 0, 0]


In [36]:
from model import *
from trainer import *

from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()

'''
from torchviz import make_dot

model = Model(all_senses = all_senses, elmo_class = elmo)
sense_embedding = model.forward(train_X[0], train_word_idx[0])
'''

'\nfrom torchviz import make_dot\n\nmodel = Model(all_senses = all_senses, elmo_class = elmo)\nsense_embedding = model.forward(train_X[0], train_word_idx[0])\n'

In [37]:
# trainer
epochs = 30
trainer = Trainer(epochs = epochs, elmo_class = elmo, all_senses = new_all_senses, all_supersenses = new_all_supersenses)

In [38]:
# get the results
# train_losses, dev_losses, dev_rs = trainer.train(train_X, train_Y, train_word_idx, dev_X, dev_Y, dev_word_idx, development = False)
# small test
train_losses, dev_losses, dev_rs = trainer.train(new_train_X, new_train_Y, new_train_idx, new_dev_X, new_dev_Y, new_dev_idx)



#############   Model Parameters   ##############
layers.word_sense.0.weight torch.Size([512, 512])
layers.word_sense.0.bias torch.Size([512])
layers.word_sense.2.weight torch.Size([300, 512])
layers.word_sense.2.bias torch.Size([300])
dimension_reduction_MLP.weight torch.Size([256, 3072])
dimension_reduction_MLP.bias torch.Size([256])
wsd_lstm.weight_ih_l0 torch.Size([1024, 256])
wsd_lstm.weight_hh_l0 torch.Size([1024, 256])
wsd_lstm.bias_ih_l0 torch.Size([1024])
wsd_lstm.bias_hh_l0 torch.Size([1024])
wsd_lstm.weight_ih_l0_reverse torch.Size([1024, 256])
wsd_lstm.weight_hh_l0_reverse torch.Size([1024, 256])
wsd_lstm.bias_ih_l0_reverse torch.Size([1024])
wsd_lstm.bias_hh_l0_reverse torch.Size([1024])
wsd_lstm.weight_ih_l1 torch.Size([1024, 512])
wsd_lstm.weight_hh_l1 torch.Size([1024, 256])
wsd_lstm.bias_ih_l1 torch.Size([1024])
wsd_lstm.bias_hh_l1 torch.Size([1024])
wsd_lstm.weight_ih_l1_reverse torch.Size([1024, 512])
wsd_lstm.weight_hh_l1_reverse torch.Size([1024, 256])
wsd_lstm.bia

Epoch: 1, Mean Training Loss: 9.431071445859711
Epoch: 1, Mean Dev Loss: 8.965237753731865
[Epoch: 2/30]


Epoch: 2, Mean Training Loss: 9.348735376336109
Epoch: 2, Mean Dev Loss: 8.909522601536342
[Epoch: 3/30]


Epoch: 3, Mean Training Loss: 9.296282664112661
Epoch: 3, Mean Dev Loss: 8.879523277282715
[Epoch: 4/30]


Epoch: 4, Mean Training Loss: 9.288850170442428
Epoch: 4, Mean Dev Loss: 8.922855922154017
[Epoch: 5/30]


...

Epoch: 26, Mean Training Loss: 9.120344123621097
Epoch: 26, Mean Dev Loss: 8.814284869602748
[Epoch: 27/30]


Epoch: 27, Mean Training Loss: 9.140803786529892
Epoch: 27, Mean Dev Loss: 8.826189722333636
[Epoch: 28/30]


Epoch: 28, Mean Training Loss: 9.132987987035992
Epoch: 28, Mean Dev Loss: 8.847142628261022
[Epoch: 29/30]


Epoch: 29, Mean Training Loss: 9.136628063245752
Epoch: 29, Mean Dev Loss: 8.838095256260463
[Epoch: 30/30]


Epoch: 30, Mean Training Loss: 9.130804500360599
Epoch: 30, Mean Dev Loss: 8.818094662257604


In [39]:
# plot the learning curve
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc

with open('train_loss.tsv', mode = 'w') as loss_file:
        
    csv_writer = csv.writer(loss_file)
    csv_writer.writerow(train_losses)

    
with open('dev_loss.tsv', mode = 'w') as loss_file:
        
    csv_writer = csv.writer(loss_file)
    csv_writer.writerow(dev_losses)

In [40]:
plt.figure(1)
rc('text', usetex = True)
rc('font', family='serif')
plt.grid(True, ls = '-.',alpha = 0.4)
plt.plot(train_losses, ms = 4, marker = 's', label = "Train Loss")
plt.legend(loc = "best")
title = "Cosine Similarity Loss (number of examples: " + str(len(new_train_X)) + ")"
plt.title(title)
plt.ylabel('Loss')
plt.xlabel('Number of Iteration')
plt.tight_layout()
plt.savefig('train_loss.png')

In [41]:
plt.figure(2)
rc('text', usetex = True)
rc('font', family='serif')
plt.grid(True, ls = '-.',alpha = 0.4)
plt.plot(dev_losses, ms = 4, marker = 'o', label = "Dev Loss")
plt.legend(loc = "best")
title = "Cosine Similarity Loss (number of examples: " + str(len(new_dev_X)) + ")"
plt.title(title)
plt.ylabel('Loss')
plt.xlabel('Number of Iteration')
plt.tight_layout()
plt.savefig('dev_loss.png')

In [43]:
# test the model
# modify the test set smaller for now
cos = nn.CosineSimilarity(dim = 0, eps = 1e-6)
correct_count = 0
        
# overall accuracy
for test_idx, test_sen in enumerate(new_test_X):
    
    test_lemma = test_sen[new_test_idx[test_idx]]
    test_emb = trainer._model.forward(test_sen, new_test_idx[test_idx])
    all_similarity = []
    
    for k, sense in enumerate(new_all_senses[test_lemma]):
        definition_vec = trainer._model.definition_embeddings[test_lemma][:, k].view(trainer._model.output_size, -1)
        cos_sim = cos(test_emb, definition_vec)
        all_similarity.append(cos_sim)
    
    test_result = all_similarity.index(max(all_similarity))
    if new_test_Y[test_idx][test_result] == 1:
        correct_count += 1

print('test size: {}'.format(len(new_test_X)))
print('overall accuracy: {}'.format(correct_count / len(new_test_X)))


test size: 9
overall accuracy: 0.5555555555555556
